In [21]:
from collections import deque

def ac3(grid):
    queue = deque([])
    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:
                for k in range(9):
                    if k != j and grid[i][k] == 0:
                        queue.append((i, k, i, j))
                    if k != i and grid[k][j] == 0:
                        queue.append((k, j, i, j))

    while queue:
        (i, j, i1, j1) = queue.popleft()
        if revise(grid, i, j, i1, j1):
            if len(grid[i][j]) == 0:
                return False
            for k in range(9):
                if k != j and grid[i][k] == 0:
                    queue.append((i, k, i, j))
                if k != i and grid[k][j] == 0:
                    queue.append((k, j, i, j))
    return True

def revise(grid, i, j, i1, j1):
    revised = False
    for digit in grid[i1][j1]:
        if digit in grid[i][j]:
            grid[i][j].remove(digit)
            revised = True
    return revised





def solve_sudoku(grid):
    if not grid:
        return None

    if not solve_with_constraints(grid):
        return None

    return grid

def solve_with_constraints(grid):
    if is_complete(grid):
        return True

    row, col = next_empty_cell(grid)
    for digit in ordered_values(grid, row, col):
        if is_valid_move(grid, row, col, digit):
            grid[row][col] = digit
            if solve_with_constraints(grid):
                return True
            grid[row][col] = 0

    return False

def next_empty_cell(grid):
    empty_cells = [(r, c) for r in range(9) for c in range(9) if grid[r][c] == 0]
    if not empty_cells:
        return None, None

    return min(empty_cells, key=lambda cell: len(possible_values(grid, cell[0], cell[1])))

def ordered_values(grid, row, col):
    return sorted(possible_values(grid, row, col), key=lambda digit: num_eliminations(grid, row, col, digit))

def possible_values(grid, row, col):
    row_values = set(grid[row])
    col_values = set(grid[r][col] for r in range(9))
    subgrid_values = set(grid[r][c] for r in range(row//3*3, row//3*3+3) for c in range(col//3*3, col//3*3+3))
    return {1, 2, 3, 4, 5, 6, 7, 8, 9} - row_values - col_values - subgrid_values

def num_eliminations(grid, row, col, digit):
    count = 0
    for r in range(9):
        if r != row and digit in possible_values(grid, r, col):
            count += 1
    for c in range(9):
        if c != col and digit in possible_values(grid, row, c):
            count += 1
    for r in range(row//3*3, row//3*3+3):
        for c in range(col//3*3, col//3*3+3):
            if (r != row or c != col) and digit in possible_values(grid, r, c):
                count += 1
    return count

def is_complete(grid):
    return all(all(cell != 0 for cell in row) for row in grid)

def is_valid_move(grid, row, col, digit):
    return digit in possible_values(grid, row, col)

def sudoku_puzzle_to_string(grid):
    return '\n'.join(' '.join(str(cell) for cell in row) for row in grid)

def string_to_sudoku_puzzle(s):
    return [[int(cell) for cell in row.split()] for row in s.strip().split('\n')]


# Example puzzle
puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

solved_puzzle = solve_sudoku(puzzle)

if solved_puzzle:
    print(sudoku_puzzle_to_string(solved_puzzle))
else:
    print("No solution exists.")

5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9


In [ ]:
import time
import random

def fitness_func(state):
    """Calculate the fitness score of a magic square state"""
    target_sum = 15
    fitness = 0
    # Check rows, columns, and diagonals
    for i in range(3):
        row_sum = sum(state[i])
        col_sum = sum(row[i] for row in state)
        diag1_sum = sum(state[i][i] for i in range(3))
        diag2_sum = sum(state[i][2-i] for i in range(3))
        fitness += abs(row_sum - target_sum) + abs(col_sum - target_sum) + abs(diag1_sum - target_sum) + abs(diag2_sum - target_sum)
    return fitness

def generate_population(pop_size, matrix):
    """Generate an initial population of magic square states with a hardcoded matrix"""
    population = []
    for _ in range(pop_size):
        state = [row[:] for row in matrix]  # Copy the matrix to avoid modifying the original
        random.shuffle(state)
        population.append(state)
    return population

def crossover(parent1, parent2):
    """Perform crossover between two parents to generate a child"""
    child = []
    for i in range(3):
        row = []
        for j in range(3):
            # Randomly choose a parent for each element
            if random.random() < 0.5:
                row.append(parent1[i][j])
            else:
                row.append(parent2[i][j])
        child.append(row)
    return child

def mutate(state, mutation_rate):
    """Perform mutation on a magic square state"""
    for i in range(3):
        for j in range(3):
            if random.random() < mutation_rate:
                # Swap the current element with a random element in the same row
                col = random.randint(0, 2)
                state[i][j], state[i][col] = state[i][col], state[i][j]
    return state

def solve_magic_square(pop_size, generations, mutation_rate, max_restarts=10):
    """Solve the magic square puzzle using a genetic algorithm with restarts"""
    matrix = [[1, 2, 3], [4, 9, 6], [7, 8, 5]]  # Hardcoded matrix
    best_solution = None
    best_fitness = float('inf')
    
    for _ in range(max_restarts):
        population = generate_population(pop_size, matrix)
        for _ in range(generations):
            new_population = []
            for _ in range(pop_size):
                parent1 = random.choice(population)
                parent2 = random.choice(population)
                child = crossover(parent1, parent2)
                child = mutate(child, mutation_rate)
                new_population.append(child)
            population = new_population
            fittest = min(population, key=fitness_func)
            current_fitness = fitness_func(fittest)
            if current_fitness < best_fitness:
                best_fitness = current_fitness
                best_solution = fittest
            if best_fitness == 0:
                return best_solution

    return best_solution

# Example usage
pop_size = 100
generations = 1000
mutation_rate = 0.2  # Increased mutation rate
max_restarts = 10

start_time = time.time()
solution = solve_magic_square(pop_size, generations, mutation_rate, max_restarts)
end_time = time.time()

print("Magic Square Solution:")
if solution is not None:
    for row in solution:
        print(row)
else:
    print("No solution found within the specified number of restarts and generations.")
print(f"Time taken: {end_time - start_time} seconds")
